<a href="https://colab.research.google.com/github/smondal13/DOE-Examples/blob/main/advanced_ticker_screener.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
!pip install finvizfinance

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 2.7 MB/s eta 0:00:00


In [8]:
import pandas as pd
import yfinance as yf
from finvizfinance.screener.overview import Overview
from typing import Optional, List, Literal, Dict
import numpy as np
from datetime import datetime

class SwingScreener:
    """
    A technical screener for swing trading that filters stocks based on
    fundamentals (via Finviz) and technical momentum/acceleration (via yfinance).
    """

    def __init__(self):
        # Configuration for data fetching
        pd.set_option('display.max_columns', None)
        pd.set_option('display.width', 1000)

    def _get_finviz_universe(
        self,
        sector: Optional[str] = None,
        industry: Optional[str] = None,
        min_market_cap_m: float = 500,
        min_avg_volume: int = 500_000,
        min_price: Optional[float] = None,
        max_price: Optional[float] = None,
        min_beta: Optional[float] = None,
        min_float: Optional[float] = None,
        rsi_below: Optional[float] = None,
        rsi_above: Optional[float] = None,
    ) -> Dict[str, str]:
        """
        Fetches a dictionary of tickers and their market caps from Finviz.
        Returns: { 'AAPL': '3000.5B', ... }
        """
        print(f"--- 1. Fetching Universe from Finviz ({sector if sector else 'All'} | {industry if industry else 'All'}) ---")

        foverview = Overview()
        filters_dict = {}

        if sector:
            filters_dict['Sector'] = sector
        if industry:
            filters_dict['Industry'] = industry

        # --- Strict Dropdown Mapping ---
        # We map numeric inputs to the nearest 'lower bound' Finviz filter to reduce result set size.

        # 1. Market Cap
        if min_market_cap_m:
            if min_market_cap_m >= 200000:
                filters_dict['Market Cap.'] = 'Mega ($200bln and more)'
            elif min_market_cap_m >= 10000:
                filters_dict['Market Cap.'] = '+Large (over $10bln)'
            elif min_market_cap_m >= 2000:
                filters_dict['Market Cap.'] = '+Mid (over $2bln)'
            elif min_market_cap_m >= 300:
                filters_dict['Market Cap.'] = '+Small (over $300mln)'
            elif min_market_cap_m >= 50:
                filters_dict['Market Cap.'] = '+Micro (over $50mln)'

        # 2. Average Volume
        if min_avg_volume:
            if min_avg_volume >= 2000000:
                filters_dict['Average Volume'] = 'Over 2M'
            elif min_avg_volume >= 1000000:
                filters_dict['Average Volume'] = 'Over 1M'
            elif min_avg_volume >= 750000:
                filters_dict['Average Volume'] = 'Over 750K'
            elif min_avg_volume >= 500000:
                filters_dict['Average Volume'] = 'Over 500K'
            elif min_avg_volume >= 200000:
                filters_dict['Average Volume'] = 'Over 200K'
            elif min_avg_volume >= 100000:
                filters_dict['Average Volume'] = 'Over 100K'
            elif min_avg_volume >= 50000:
                filters_dict['Average Volume'] = 'Over 50K'

        # 3. Price
        if min_price:
            if min_price >= 50:
                filters_dict['Price'] = 'Over $50'
            elif min_price >= 20:
                filters_dict['Price'] = 'Over $20'
            elif min_price >= 10:
                filters_dict['Price'] = 'Over $10'
            elif min_price >= 5:
                filters_dict['Price'] = 'Over $5'
            elif min_price >= 1:
                filters_dict['Price'] = 'Over $1'

        # 4. Beta
        if min_beta is not None:
            if min_beta >= 4: filters_dict['Beta'] = 'Over 4'
            elif min_beta >= 3: filters_dict['Beta'] = 'Over 3'
            elif min_beta >= 2.5: filters_dict['Beta'] = 'Over 2.5'
            elif min_beta >= 2: filters_dict['Beta'] = 'Over 2'
            elif min_beta >= 1.5: filters_dict['Beta'] = 'Over 1.5'
            elif min_beta >= 1: filters_dict['Beta'] = 'Over 1'
            elif min_beta >= 0.5: filters_dict['Beta'] = 'Over 0.5'
            elif min_beta >= 0: filters_dict['Beta'] = 'Over 0'

        # 5. Float
        if min_float:
            if min_float >= 100_000_000: filters_dict['Float'] = 'Over 100M'
            elif min_float >= 50_000_000: filters_dict['Float'] = 'Over 50M'
            elif min_float >= 20_000_000: filters_dict['Float'] = 'Over 20M'
            elif min_float >= 10_000_000: filters_dict['Float'] = 'Over 10M'
            elif min_float >= 5_000_000: filters_dict['Float'] = 'Over 5M'
            elif min_float >= 2_000_000: filters_dict['Float'] = 'Over 2M'
            elif min_float >= 1_000_000: filters_dict['Float'] = 'Over 1M'

        # 6. RSI (14) - NEW
        # Finviz only allows one filter selection for RSI (Over X OR Under Y).
        # We prioritize "Below" if provided (Oversold), otherwise "Above" (Momentum).
        if rsi_below is not None:
            if rsi_below <= 30: filters_dict['RSI (14)'] = 'Oversold (30)'
            elif rsi_below <= 40: filters_dict['RSI (14)'] = 'Oversold (40)'
            elif rsi_below <= 50: filters_dict['RSI (14)'] = 'Not Overbought (<60)' # Loose approximation
            elif rsi_below <= 60: filters_dict['RSI (14)'] = 'Not Overbought (<60)'
            elif rsi_below <= 70: filters_dict['RSI (14)'] = 'Not Overbought (<50)' # Typo in finviz logic check, using 60 is safer
            # Custom mappings for exact keys:
            if rsi_below <= 20: filters_dict['RSI (14)'] = 'Oversold (20)'
            elif rsi_below <= 30: filters_dict['RSI (14)'] = 'Oversold (30)'
            elif rsi_below <= 40: filters_dict['RSI (14)'] = 'Oversold (40)'
            elif rsi_below <= 50: filters_dict['RSI (14)'] = 'Not Overbought (<60)' # Safe bet

        elif rsi_above is not None:
            if rsi_above >= 80: filters_dict['RSI (14)'] = 'Overbought (80)'
            elif rsi_above >= 70: filters_dict['RSI (14)'] = 'Overbought (70)'
            elif rsi_above >= 60: filters_dict['RSI (14)'] = 'Overbought (60)'
            elif rsi_above >= 50: filters_dict['RSI (14)'] = 'Not Oversold (>50)'
            elif rsi_above >= 40: filters_dict['RSI (14)'] = 'Not Oversold (>40)'
            elif rsi_above >= 30: filters_dict['RSI (14)'] = 'Over 30'

        print(f"Applying Finviz Filters: {filters_dict}")

        # Set filters
        try:
            foverview.set_filter(filters_dict=filters_dict)
            df_finviz = foverview.screener_view()
        except Exception as e:
            print(f"Finviz Filter Error (could be empty result or connection): {e}")
            return {}

        if df_finviz.empty:
            print("No tickers found matching Finviz criteria.")
            return {}

        # Filter Price locally for precision
        if min_price or max_price:
            df_finviz['Price'] = pd.to_numeric(df_finviz['Price'], errors='coerce')
            if min_price:
                df_finviz = df_finviz[df_finviz['Price'] >= min_price]
            if max_price:
                df_finviz = df_finviz[df_finviz['Price'] <= max_price]

        # Extract Ticker AND Market Cap to avoid re-fetching
        # Creates a dictionary: {'AAPL': '3000B', 'MSFT': '2800B'}
        tickers_map = dict(zip(df_finviz['Ticker'], df_finviz['Market Cap']))

        print(f"Found {len(tickers_map)} potential candidates in {sector}/{industry}.")
        return tickers_map

    def _calculate_indicators(self, df: pd.DataFrame, price_oc_fluc_day: int = 14) -> pd.DataFrame:
        """
        Computes Moving Averages, Momentum, Acceleration, RSI, and Consolidation Fluctuation.
        """
        if len(df) < 200:
            return df

        # 1. Calculate Moving Averages
        df['ema_8'] = df['Close'].ewm(span=8, adjust=False).mean()
        df['ema_20'] = df['Close'].ewm(span=20, adjust=False).mean()
        df['sma_50'] = df['Close'].rolling(window=50).mean()
        df['sma_200'] = df['Close'].rolling(window=200).mean()

        # 2. Calculate Momentum (Normalized Rate of Change)
        for ma in ['ema_8', 'ema_20', 'sma_50', 'sma_200']:
            # Momentum = % change of the MA line
            df[f'{ma}_momentum'] = df[ma].pct_change() * 100

            # Acceleration = Change in Momentum (First derivative of momentum)
            df[f'{ma}_acceleration'] = df[f'{ma}_momentum'].diff()

        # 3. Calculate RSI (14)
        delta = df['Close'].diff()
        gain = (delta.where(delta > 0, 0)).fillna(0)
        loss = (-delta.where(delta < 0, 0)).fillna(0)

        avg_gain = gain.ewm(com=13, adjust=False).mean()
        avg_loss = loss.ewm(com=13, adjust=False).mean()

        rs = avg_gain / avg_loss
        df['rsi'] = 100 - (100 / (1 + rs))

        # 4. Consolidation Fluctuation (Based on Open/Close Bodies)
        # Identifies squeezes by ignoring wicks and looking at where value was accepted
        if price_oc_fluc_day:
            # Create a Series for daily Max of (Open, Close) and Min of (Open, Close)
            daily_max_oc = df[['Open', 'Close']].max(axis=1)
            daily_min_oc = df[['Open', 'Close']].min(axis=1)

            # Find the Period Max and Min over N days
            period_max = daily_max_oc.rolling(window=price_oc_fluc_day).max()
            period_min = daily_min_oc.rolling(window=price_oc_fluc_day).min()

            # Calculate % difference
            df['consolidation_fluc_pct'] = ((period_max - period_min) / period_min) * 100

        return df

    def _format_macro_number(self, num) -> str:
        """
        Formats large floats (e.g., 2.5e9) into readable strings (e.g., '2.50B').
        """
        if pd.isna(num) or num == "N/A":
            return "N/A"
        try:
            val = float(num)
        except ValueError:
            return str(num)

        if val >= 1e12:
            return f"{val/1e12:.2f}T"
        elif val >= 1e9:
            return f"{val/1e9:.2f}B"
        elif val >= 1e6:
            return f"{val/1e6:.2f}M"
        elif val >= 1e3:
            return f"{val/1e3:.2f}K"
        return f"{val:.2f}"

    def run_screen(
        self,
        sector: Optional[str] = "Technology",
        industry: Optional[str] = "Biotechnology",
        candle: str = "1d",
        min_price: Optional[float] = None,
        max_price: Optional[float] = None,
        min_float: Optional[float] = None,
        max_float: Optional[float] = None,
        min_market_cap_m: float = 500,
        min_avg_volume: int = 500_000,
        min_beta: float = 1.0,
        min_gap_pct: Optional[float] = None,
        min_change_pct: Optional[float] = None,
        price_above_sma200: bool = True,
        rsi_below: Optional[float] = None,
        rsi_above: Optional[float] = None,
        price_oc_fluc_day: int = 14, # Added: Fluctuation lookback period
        sma200_momentum: Optional[Literal["up", "down"]] = None,
        sma50_momentum: Optional[Literal["up", "down"]] = None,
        ema_8_momentum: Optional[Literal["up", "down"]] = None,
        ema_20_momentum: Optional[Literal["up", "down"]] = None,
        save_csv: bool = False,
        save_tickers_txt: bool = True
    ) -> pd.DataFrame:

        # 1. Fetch Universe (returns dict of ticker -> market_cap)
        tickers_map = self._get_finviz_universe(
            sector=sector,
            industry=industry,
            min_price=min_price,
            max_price=max_price,
            min_market_cap_m=min_market_cap_m,
            min_avg_volume=min_avg_volume,
            min_beta=min_beta,
            min_float=min_float,
            rsi_below=rsi_below,
            rsi_above=rsi_above
        )

        if not tickers_map:
            return pd.DataFrame()

        print(f"--- 2. Downloading & processing Technicals for {len(tickers_map)} tickers ---")

        valid_data = []
        yf_period = "2y"
        if candle in ["1h", "4h"]:
            yf_period = "729d" # max for hourly data in yf

        # Iterate through dictionary keys (tickers)
        for ticker in tickers_map:
            try:
                # Threading is False to prevent rate limiting issues with YF
                df = yf.download(ticker, period=yf_period, interval=candle, progress=False, threads=False, auto_adjust=True)

                if df.empty or len(df) < 201:
                    continue

                # Flatten MultiIndex columns if present
                if isinstance(df.columns, pd.MultiIndex):
                    df.columns = df.columns.get_level_values(0)

                # --- Fundamental Checks (Volume) ---
                # Check 20-day average volume
                if len(df) >= 20:
                    avg_vol = df['Volume'].rolling(20).mean().iloc[-1]
                    if avg_vol < min_avg_volume:
                        continue
                else:
                    continue

                # --- Gap Check ---
                if min_gap_pct:
                    prev_close = df['Close'].shift(1).iloc[-1]
                    curr_open = df['Open'].iloc[-1]
                    gap = ((curr_open - prev_close) / prev_close) * 100
                    if gap < min_gap_pct:
                        continue

                # --- Change Check ---
                if min_change_pct:
                    prev_close = df['Close'].shift(1).iloc[-1]
                    curr_close = df['Close'].iloc[-1]
                    change = ((curr_close - prev_close) / prev_close) * 100
                    if change < min_change_pct:
                        continue

                # --- Calculate Technicals ---
                df = self._calculate_indicators(df, price_oc_fluc_day=price_oc_fluc_day)

                # Critical Safety Check: Ensure columns exist before filtering
                required_cols = ['sma_200_momentum', 'sma_50_momentum', 'ema_20_momentum', 'ema_8_momentum', 'rsi']
                if price_oc_fluc_day:
                    required_cols.append('consolidation_fluc_pct')

                if not all(col in df.columns for col in required_cols):
                    # print(f"Skipping {ticker}: Insufficient history for indicators")
                    continue

                # Get the latest row
                current = df.iloc[-1]

                # --- Technical Filters ---
                if price_above_sma200:
                    if current['Close'] < current['sma_200']:
                        continue

                # RSI Filters
                if rsi_below is not None:
                    if current['rsi'] > rsi_below:
                        continue

                if rsi_above is not None:
                    if current['rsi'] < rsi_above:
                        continue

                # Momentum Direction Filters
                if sma200_momentum:
                    if sma200_momentum == "up" and current['sma_200_momentum'] <= 0: continue
                    if sma200_momentum == "down" and current['sma_200_momentum'] >= 0: continue

                if sma50_momentum:
                    if sma50_momentum == "up" and current['sma_50_momentum'] <= 0: continue
                    if sma50_momentum == "down" and current['sma_50_momentum'] >= 0: continue

                if ema_8_momentum:
                    if ema_8_momentum == "up" and current['ema_8_momentum'] <= 0: continue
                    if ema_8_momentum == "down" and current['ema_8_momentum'] >= 0: continue

                if ema_20_momentum:
                    if ema_20_momentum == "up" and current['ema_20_momentum'] <= 0: continue
                    if ema_20_momentum == "down" and current['ema_20_momentum'] >= 0: continue

                # If we passed all filters, add to list
                row_data = {
                    'ticker_name': ticker,
                    'market_cap': tickers_map.get(ticker, "N/A"),
                    'price': round(current['Close'], 2),
                    'rsi': round(current['rsi'], 2),
                    'ema_8_momentum': round(current['ema_8_momentum'], 4),
                    'ema_8_acceleration': round(current['ema_8_acceleration'], 4),
                    'ema_20_momentum': round(current['ema_20_momentum'], 4),
                    'ema_20_acceleration': round(current['ema_20_acceleration'], 4),
                    'sma_50_momentum': round(current['sma_50_momentum'], 4),
                    'sma_50_acceleration': round(current['sma_50_acceleration'], 4),
                    'sma_200_momentum': round(current['sma_200_momentum'], 4),
                    'sma_200_acceleration': round(current['sma_200_acceleration'], 4),
                }

                # Add consolidation pct if requested
                if price_oc_fluc_day and 'consolidation_fluc_pct' in current:
                    row_data['consolidation_fluc_pct'] = round(current['consolidation_fluc_pct'], 2)

                valid_data.append(row_data)

            except Exception as e:
                print(f"Error processing {ticker}: {e}")
                continue

        results_df = pd.DataFrame(valid_data)

        # Output Handling
        if not results_df.empty:
            # --- Ranking / Sorting ---
            # Calculate a 'Trend Score' to rank the best setups
            # Weighting: EMA8 (Immediate) > EMA20 > SMA50 > SMA200
            results_df['Trend_Score'] = (
                (results_df['ema_8_momentum'] * 2.0) +
                (results_df['ema_20_momentum'] * 1.5) +
                (results_df['sma_50_momentum'] * 1.0) +
                (results_df['sma_200_momentum'] * 0.5)
            )

            # Sort by highest score first
            results_df = results_df.sort_values(by='Trend_Score', ascending=False)

            # Format Market Cap for readability (Issues: Sorting becomes alphabetical, not numerical)
            if 'market_cap' in results_df.columns:
                results_df['market_cap'] = results_df['market_cap'].apply(self._format_macro_number)

            print(f"\n--- Results: Found {len(results_df)} tickers (Sorted by Trend Score) ---")
            print(results_df.head())

            timestamp = datetime.now().strftime("%Y%m%d_%H%M")

            if save_csv:
                filename = f"screen_results_{sector}_{industry}_{timestamp}.csv"
                results_df.to_csv(filename, index=False)
                print(f"Saved CSV to {filename}")

            if save_tickers_txt:
                filename_txt = f"tickers_{sector}_{industry}_{timestamp}.txt"
                with open(filename_txt, 'w') as f:
                    for t in results_df['ticker_name']:
                        f.write(f"{t}\n")
                print(f"Saved Ticker List to {filename_txt}")
        else:
            print("No tickers matched the specific Technical criteria.")

        return results_df

# --- Usage Example ---
if __name__ == "__main__":
    screener = SwingScreener()

    df_results = screener.run_screen(
        # 1. Universe Selection
        sector = None,                   # e.g., "Healthcare", or None for All
        industry = None,                 # e.g., "Biotechnology", or None for All

        # 2. Fundamental Filters
        min_market_cap_m = 200_000,
        min_avg_volume = 200_000,
        min_float = 20_000_000,         # Now active: Float > 20M
        max_float = None,
        min_price = 5.0,
        max_price = None,
        min_beta = 1.0,                 # Now active: Beta > 1

        # 3. Timeframe
        candle = "1d",

        # 4. Price Action Filters
        min_gap_pct = None,
        min_change_pct = None,
        price_above_sma200 = True,

        # 5. Consolidation / Fluctuation (New)
        price_oc_fluc_day = 14,         # Calculates % range of Open/Close over last 14 days

        # 6. RSI Filters
        rsi_below = None,               # e.g., 30 for oversold (applies Finviz "Oversold (30)")
        rsi_above = None,               # e.g., 50 for bullish momentum (applies Finviz "Not Oversold (>50)")

        # 7. Momentum Filters
        ema_8_momentum = "up",
        ema_20_momentum = "up",
        sma50_momentum = None,
        sma200_momentum = "up",

        # 8. Output Options
        save_csv = True,
        save_tickers_txt = True
    )

--- 1. Fetching Universe from Finviz (All | All) ---
Applying Finviz Filters: {'Market Cap.': 'Mega ($200bln and more)', 'Average Volume': 'Over 200K', 'Price': 'Over $5', 'Beta': 'Over 1', 'Float': 'Over 20M'}
Found 32 potential candidates in None/None.
--- 2. Downloading & processing Technicals for 32 tickers ---


/tmp/ipython-input-1938196464.py:288: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, period=yf_period, interval=candle, progress=False, threads=False)
/tmp/ipython-input-1938196464.py:288: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, period=yf_period, interval=candle, progress=False, threads=False)
/tmp/ipython-input-1938196464.py:288: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, period=yf_period, interval=candle, progress=False, threads=False)
/tmp/ipython-input-1938196464.py:288: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, period=yf_period, interval=candle, progress=False, threads=False)
/tmp/ipython-input-1938196464.py:288: FutureWarning: YF.download() has changed argument auto_adjust default to True
  df = yf.download(ticker, period=yf_period, int


--- Results: Found 11 tickers (Sorted by Trend Score) ---
   ticker_name market_cap   price    rsi  ema_8_momentum  ema_8_acceleration  ema_20_momentum  ema_20_acceleration  sma_50_momentum  sma_50_acceleration  sma_200_momentum  sma_200_acceleration  consolidation_fluc_pct  Trend_Score
10         WFC    291.18B   92.76  71.83          0.7560             -0.1703           0.5500              -0.0410           0.2985               0.0118            0.1121               -0.0001                   12.07      2.69155
2            C    200.04B  111.80  75.61          0.6580             -0.1792           0.5979              -0.0612           0.2991               0.0275            0.2010                0.0011                   14.76      2.61245
9         TSLA      1.53T  458.96  60.18          0.7675              0.5576           0.5024               0.2378           0.1055               0.1633            0.2564                0.0298                   14.12      2.52230
0          AXP    263